# PyTao advanced usage

In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12,8)
%config InlineBackend.figure_format = 'retina'

In [ ]:
from pytao import Tao, TaoModel, util, run_tao
from pytao.tao_ctypes.evaluate import evaluate_tao
import os
?TaoModel

## Inititialize Tao

In [ ]:
# Point to local installation
BASE_DIR=os.environ['ACC_ROOT_DIR'] 
print('Bmad installation: ', BASE_DIR)

# Pick an example init
root = os.path.join(BASE_DIR, 'tao/examples/csr_beam_tracking/')

INPUT_FILE = root+'tao.init'
assert os.path.exists(INPUT_FILE), INPUT_FILE

## run_tao

In [ ]:
M=run_tao(input_file=INPUT_FILE, ploton=True)
M

## evaluate_tao

In [ ]:
?evaluate_tao

In [ ]:
res = evaluate_tao(settings={'csr_param:write_csr_wake':True,
                            'csr_param:ds_track_step':0.001}, 
                   input_file=root+'tao.init', 
                   run_commands=['set global track_type=beam'],
                   expressions = ['lat::orbit.x[FF.PIP02A]', 'beam::norm_emit.x[end]'],
                   ploton=False, 
                   archive_csr_wake=True,
                   beam_archive_path = '.')
res

In [ ]:
from pmd_beamphysics import ParticleGroup, particle_paths
from h5py import File
afile = res['beam_archive']
h5 = File(afile, 'r')
ppaths = particle_paths(h5)
ppaths

In [ ]:
list(h5)

In [ ]:
P = ParticleGroup(h5[ppaths[-1]])
P.plot('delta_t', 'delta_pz', bins=200)

In [ ]:
os.remove(afile)

## evaluate examples

In [ ]:
M.evaluate('lat::orbit.x[end]')

## csr_wake diagnostic file

In [ ]:
from pytao.misc.csr import read_csr_wake_data_h5, process_csr_wake_data
import numpy as np

In [ ]:
cdat = read_csr_wake_data_h5(h5, 'csr_wake')

In [ ]:
cdat.keys()

In [ ]:
# This is the actual data
dat = cdat['3:FF.BEN01']['data']
dat.shape

In [ ]:
# This will process into a more useful form for data analysis
pdat = process_csr_wake_data(cdat)

?process_csr_wake_data

In [ ]:
from pytao.misc.csr_plot import plot_csr_wake, plot_csr_stats
from pytao.misc.csr import csr_wake_stats_at_step

In [ ]:
from ipywidgets import interact
def plot1(step=0):
    plot_csr_wake(pdat, step=step)
nstep = len(pdat['z'])
interact(plot1, step=(0, nstep-1, 1) )

In [ ]:
# Another convenience plot
plot_csr_stats(pdat)  